In [1]:
import os 
import sys

import sklearn as sk
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt 
import xgboost as xgb

from scipy.stats import kurtosis, entropy
from scipy.fft import fft

sys.path.append(r"c:\Users\keng-tse\Desktop\csi_tool") #這行為絕對路徑，如需使用，必須要修改為當前決路徑
import csi_tool
import denoise 

In [8]:
from statsmodels.multivariate.manova import MANOVA
import pandas as pd
import numpy as np

In [2]:
base_path = r"C:\Users\keng-tse\Desktop\csi_tool\csi_dataset\localization_phone\1207_phone\20Mhz\csv\CIR"
base_path2 = r"C:\Users\keng-tse\Desktop\csi_tool\csi_dataset\localization_phone\1205_phone\20Mhz\csv"
base_path = r"C:\Users\keng-tse\Desktop\csi_tool\csi_dataset\localization_phone\1201_phone\20MHz\csv\"
reference_points = {}

# 每個點的實際位置 (x, y)，基於 0.6m 的間距
spacing = 0.6  # 每隔 0.6m
coordinates = {
    # 下邊界 (1-10 和 40-31)
    1: (0, 0), 40: (0.6, 0), 39: (1.2, 0), 38: (1.8, 0), 37: (2.4, 0),
    36: (3.0, 0), 35: (3.6, 0), 34: (4.2, 0), 33: (4.8, 0), 32: (5.4, 0), 31: (6.0, 0),

    # 左邊界 (1-11)
    2: (0, 0.6), 3: (0, 1.2), 4: (0, 1.8), 5: (0, 2.4),
    6: (0, 3.0), 7: (0, 3.6), 8: (0, 4.2), 9: (0, 4.8), 10: (0, 5.4), 11: (0, 6.0),

    # 上邊界 (11-21)
    12: (0.6, 6.0), 13: (1.2, 6.0), 14: (1.8, 6.0), 15: (2.4, 6.0),
    16: (3.0, 6.0), 17: (3.6, 6.0), 18: (4.2, 6.0), 19: (4.8, 6.0),
    20: (5.4, 6.0), 21: (6.0, 6.0),

    # 右邊界 (21-31)
    22: (6.0, 5.4), 23: (6.0, 4.8), 24: (6.0, 4.2), 25: (6.0, 3.6),
    26: (6.0, 3.0), 27: (6.0, 2.4), 28: (6.0, 1.8), 29: (6.0, 1.2), 30: (6.0, 0.6),

    # 中間點 (41-49)
    41: (3.0, 0.6), 42: (3.0, 1.2), 43: (3.0, 1.8),
    44: (3.0, 2.4), 45: (3.0, 3.0), 46: (3.0, 3.6),
    47: (3.0, 4.2), 48: (3.0, 4.8), 49: (3.0, 5.4)
}

# 生成 reference_points 字典
for ref_id, coord in coordinates.items():
    folder_path = os.path.join(base_path, f"reference_point_{ref_id}.xlsx")
    reference_points[folder_path] = (ref_id, coord)

# 打印結果
for path, (ref_id, coord) in reference_points.items():
    print(f"Path: {path}, ID: {ref_id}, Coordinates: {coord}")


Path: C:\Users\keng-tse\Desktop\csi_tool\csi_dataset\localization_phone\1207_phone\20Mhz\csv\CIR\reference_point_1.xlsx, ID: 1, Coordinates: (0, 0)
Path: C:\Users\keng-tse\Desktop\csi_tool\csi_dataset\localization_phone\1207_phone\20Mhz\csv\CIR\reference_point_40.xlsx, ID: 40, Coordinates: (0.6, 0)
Path: C:\Users\keng-tse\Desktop\csi_tool\csi_dataset\localization_phone\1207_phone\20Mhz\csv\CIR\reference_point_39.xlsx, ID: 39, Coordinates: (1.2, 0)
Path: C:\Users\keng-tse\Desktop\csi_tool\csi_dataset\localization_phone\1207_phone\20Mhz\csv\CIR\reference_point_38.xlsx, ID: 38, Coordinates: (1.8, 0)
Path: C:\Users\keng-tse\Desktop\csi_tool\csi_dataset\localization_phone\1207_phone\20Mhz\csv\CIR\reference_point_37.xlsx, ID: 37, Coordinates: (2.4, 0)
Path: C:\Users\keng-tse\Desktop\csi_tool\csi_dataset\localization_phone\1207_phone\20Mhz\csv\CIR\reference_point_36.xlsx, ID: 36, Coordinates: (3.0, 0)
Path: C:\Users\keng-tse\Desktop\csi_tool\csi_dataset\localization_phone\1207_phone\20Mhz\csv

In [3]:
def load_data(reference_points):
    data = []          # 用于存储所有数据
    rp_labels = []     # 用于存储参考点编号标签
    coord_labels = []  # 用于存储 (x, y) 坐标标签

    for path, (ref_id, coord) in reference_points.items():
        df = pd.read_excel(path)
        data.append(df.values)
        rp_labels.extend([ref_id] * len(df))
        coord_labels.extend([coord] * len(df))
    data = pd.DataFrame(np.vstack(data))
    rp_labels = pd.Series(rp_labels, name="Reference Point ID")
    coord_labels = pd.DataFrame(coord_labels, columns=["X", "Y"])
    return data, rp_labels, coord_labels

In [4]:
data, rp_labels, coord_labels = load_data(reference_points)

In [10]:
amp_data = data.iloc[:, :48]  
phase_data = data.iloc[:, 48:] 

amp_data = denoise.hampel_filter(np.array(amp_data))
amp_data = denoise.moving_median_filter(amp_data, 5)
#amp_data = np.concatenate([amp_data, phase_data], axis=1)
amp_data = pd.DataFrame(amp_data)

In [11]:
# 修正警告：用 .loc 明確添加列
amp_data_c = amp_data.copy()  # 防止修改原始 DataFrame
amp_data_c.loc[:, 'label'] = rp_labels


amp_data_c.columns = [f"sub_{col}" if isinstance(col, int) else col for col in amp_data_c.columns]

# 構建公式
columns = amp_data_c.columns[:-1]  # 所有子載波作為自變量
formula = ' + '.join(columns)    # 使用 '+' 連接所有子載波
formula = f"{formula} ~ label"   # 子載波 vs 參考點標籤

# 執行 MANOVA 測試
manova = MANOVA.from_formula(formula, data=amp_data_c)
result = manova.mv_test()

# 打印結果
print(result)



                     Multivariate linear model
                                                                    
--------------------------------------------------------------------
       Intercept         Value   Num DF   Den DF    F Value   Pr > F
--------------------------------------------------------------------
          Wilks' lambda  0.0346 48.0000 24326.0000 14127.3740 0.0000
         Pillai's trace  0.9654 48.0000 24326.0000 14127.3740 0.0000
 Hotelling-Lawley trace 27.8761 48.0000 24326.0000 14127.3740 0.0000
    Roy's greatest root 27.8761 48.0000 24326.0000 14127.3740 0.0000
--------------------------------------------------------------------
                                                                    
--------------------------------------------------------------------
             label          Value   Num DF   Den DF   F Value Pr > F
--------------------------------------------------------------------
              Wilks' lambda 0.9099 48.0000 24326.0000 50